In [1]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
import pandas as pd
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm

%matplotlib inline

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
from keras.models import load_model
from efficientnet import *

IMG_SIZE = 224
effnet = EfficientNetB3(input_shape=(IMG_SIZE,IMG_SIZE, 3), weights=None, include_top=False)

def build_model():
    model = Sequential()
    model.add(effnet)
    model.add(layers.GlobalAveragePooling2D())
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(1024))
#     model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(5, activation='sigmoid'))
    return model

model = build_model()

for layer in model.layers:
    layer.trainable = True

model.load_weights('../input/efficientnet-new-kfold-3/best_QWK_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b3 (Model)      (None, 7, 7, 1536)        10783528  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 7685      
Total params: 10,791,213
Trainable params: 10,703,917
Non-trainable params: 87,296
_________________________________________________________________


In [3]:
submit = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [4]:
def preprocess_image(image_path, desired_size=224):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (desired_size,desired_size))
    img = cv2.addWeighted(img,4,cv2.GaussianBlur(img, (0,0), desired_size/40) ,-4 ,128)
    return img
    
predicted = []
submit = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
SIZE = 224

for i, name in tqdm(enumerate(submit['id_code'])):
    path = os.path.join('../input/aptos2019-blindness-detection/test_images/', name+'.png')
#     image = cv2.imread(path)
    img = preprocess_image(path, desired_size=224)
#     image = cv2.resize(image, (SIZE, SIZE))
    y_test = model.predict((img[np.newaxis])/255) > 0.5
    y_test = y_test.astype(int).sum(axis=1) - 1
    #score_predict = model.predict((img[np.newaxis])/255)
    #label_predict = np.argmax(score_predict)
    # label_predict = score_predict.astype(int).sum() - 1
    predicted.append(str(y_test[0]))

1928it [01:35, 20.11it/s]


In [5]:
predicted

['2',
 '3',
 '2',
 '2',
 '2',
 '2',
 '3',
 '1',
 '3',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '3',
 '2',
 '0',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '1',
 '3',
 '3',
 '2',
 '2',
 '3',
 '1',
 '2',
 '2',
 '3',
 '2',
 '2',
 '2',
 '2',
 '1',
 '0',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '3',
 '2',
 '1',
 '3',
 '2',
 '3',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '1',
 '2',
 '3',
 '2',
 '0',
 '2',
 '2',
 '1',
 '1',
 '3',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '0',
 '0',
 '0',
 '2',
 '2',
 '2',
 '3',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '0',
 '0',
 '3',
 '2',
 '3',
 '3',
 '2',
 '2',
 '0',
 '2',
 '0',
 '2',
 '3',
 '1',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '0',
 '1',
 '0',
 '2',
 '1',
 '2',
 '2',
 '1',
 '2',
 '1',
 '2',
 '3',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '3',
 '0'

In [6]:
submit['diagnosis'] = predicted
submit.to_csv('submission.csv', index=False)
submit.head()

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,3
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,2
